### SHORT DESCRIPTION

* NCORE - numbers of core. For parallel pipeline NCORE > 1 (max NCORE = 8)
* DELETE_NOW - If True, bad reads (by filters) will be deleted after each filter
* IS_SHORT_FLANK - If True, ignore additional PRIMER and AD1 in R1
* CHAOS - If True, you're not sure that file(R1) is contained R1 and file(R2) is contained R2
* MIN_READ - Create own FIX and dbRIP file for each library (if NUM_READS >= MIN_READ)
* INSWINDOW_FIX - Delete reads if CURRENT_POS ~ [RESTRICT_SITE : IS + INSWINDOWS_FIX] (for INS_STRAND = +)
* SHIFT_RESTRICT_SITE - Mark reads if RESTRICT_SITE was found in [RESTRICT_SITE :  IS + SHIFT] (for INS_STRAND = +)
* SHIFT_MISS_PRIMER - Mark reads if PRIMER was found in [IS :  IS + SHIFT] (for INS_STRAND = +)
* SHIFT_MISS_RE -  Mark reads if PRIMER was found in [IS :  IS + SHIFT] (for INS_STRAND = +)
* RE_HAMMING - Delete reads RE_HAMMING_CURRENT >= RE_HAMMING
* FLANK_ERRORS - Delete reads FLANK_ERRORS_CURRENT >= FLANK_ERRORS
* REPEAT - Delete reads REPEAT_CURRENT >= REPEAT
* MISS_PRIMER - Delete reads MISS_PRIMER_CURRENT <= MISS_PRIMER
* MISS_RE - Delelte reads MISS_RE_CURRENT <= MISS_RE
* TEMPLATE_SWITCH_MD - Delete reads MDMATCH >= TEMPLATE_SWITCH_MD
* WINDOW - Window for megaclustering
* MAIN_FLAN_LEN - For template switch. TARGET_READS = PRIMER + RE + MAIN_FLAN_LEN(of R1)

In [1]:
#LIBRARY
import imp
import os

#PATHWAYS
INPUTDIR = ''
OUTPUTDIR = ''
MAPPER = 'bwa' #bwa or bowtie

#SYS PARAMETRS
NCORE = 1

#PREPROCESSING
PRIMER = ''
SHIFT = 5
MIST = 1
AD1 = ''
AD2 = ''
BLEN = 
RE = ''
RESTRICT_SITE = ''
R2_START = ''
IS_SHORT_FLANK = False
CHAOS = False
TRIMM_N = 0
SKIP_SHORT_READS = 200
MID_MIST_SHORT_READS = (1, 2) # (r1 ,r2)
END_MIST_SHORT_READS = (5, 5) # (r1 ,r2)
PLACE_OF_SEARCH_TAIL = (None, None) # (r1 ,r2)
MIN_SEQ_LEN_AFTER_TRIMM = (0, 0) # (r1 ,r2)
  
#FILTERS
FIX_WINDOW = 0
MIN_READ = 1
INSWINDOW_FIX = 40
SHIFT_RESTRICT_SITE = 3 
SHIFT_MISS_PRIMER = 12
SHIFT_MISS_RE = 8

#THRESHOLDS
RE_HAMMING = 2
FLANK_ERRORS = 5
REPEAT = 0.8
MISS_PRIMER = 4
MISS_RE = 2
TEMPLATE_SWITCH_MD = 30

#METACLUSTERING
WINDOW = 100
MAIN_FLANK_LEN = 11

SyntaxError: invalid syntax (<ipython-input-1-10d1fad7b2b0>, line 19)

### <font color='blue'>1st round</font> 

#### 1. Raw FASTQ files preprocessing

In [ ]:
import imp
import trimmALU
imp.reload(trimmALU)

trimmALU.main(inputdir = INPUTDIR,
              outputdir = os.path.abspath(OUTPUTDIR) + '/preprocessing/',
              primer = PRIMER,
              ad1 = AD1,
              ad2 = AD2,
              blen = BLEN,
              shift = SHIFT,
              mist = MIST,
              restrict_site = RESTRICT_SITE,
              re_part = RE,
              r2_start = R2_START,
              is_short_flank = IS_SHORT_FLANK,
              chaos = CHAOS,
              n_core = NCORE,
              trimm_n = TRIMM_N,
              skip_short_reads = SKIP_SHORT_READS,
              mid_mist_short_reads = MID_MIST_SHORT_READS,
              end_mist_short_reads = END_MIST_SHORT_READS,
              place_of_search_tail=PLACE_OF_SEARCH_TAIL,
              min_seq_len_after_trimm=MIN_SEQ_LEN_AFTER_TRIMM)

#### 2. Map FASTQ by BWA

In [ ]:
import bwamem
imp.reload(bwamem)

if MAPPER == 'bowtie':
    mapper_execline = 'bowtie2 -p 4'
    refway = '/raid/retroelement/test/reference_genome/hg38_indexed_bowtie2_rebuild/hg38'
else:
    if MAPPER != 'bwa':
        print('BWA')
    mapper_execline = './bwa-0.7.13/bwa mem -t 4'
    refway = '/raid/retroelement/test/reference_genome/hg38_indexed_bwa/hg38'

bwamem.main(inputdir = os.path.abspath(OUTPUTDIR) + '/preprocessing/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/mapping/',
            refway = refway,
            bwaline = mapper_execline)

#### 3. Filter SAM files

In [ ]:
import samfilter_pysam
imp.reload(samfilter_pysam)

samfilter.main(inputdir = os.path.abspath(OUTPUTDIR) + '/mapping/',
               outputdir = os.path.abspath(OUTPUTDIR) + '/table/',
               n_core = NCORE)

#### 4. Collapse reads by (chr, strand, pos)

In [ ]:
import collapser
imp.reload(collapser)

collapser.main(inputdir = os.path.abspath(OUTPUTDIR) + '/table/',
               outputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
               target_re = RE,
               n_core = 1)

#### 5. Create own fix and polymorph ALUBASE

In [ ]:
import exactmatch_fpALU
imp.reload(exactmatch_fpALU)

exactmatch_fpALU.main(inputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                      outputdir = os.path.abspath(OUTPUTDIR) + '/ematch_table/',
                      replibrary = 'data/FixAndPolymorphALU2' + RESTRICT_SITE + '_hg38_autosomeXY.txt.bz2',
                      min_read = MIN_READ,
                      n_core = NCORE,
                      window = FIX_WINDOW)

#### 6. Count NUM_READS, NUM_BC, TLEN for each fix and polymorph ALUBASE

In [ ]:
import ematch_fixbc
imp.reload(ematch_fixbc)

ematch_fixbc.main(inputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                  outputdir = os.path.abspath(OUTPUTDIR) + '/ematch_fixbc/',
                  replibrary = 'data/FixAndPolymorphALU2' + RESTRICT_SITE + '_hg38_autosomeXY.txt.bz2',
                  min_read = MIN_READ,
                  n_core = NCORE)

#### 7. Delete fix and polymorph ALU from reads

In [ ]:
import intersection_fpALU
imp.reload(intersection_fpALU)

intersection_fpALU.main(inputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                        outputdir = os.path.abspath(OUTPUTDIR) + '/notfpALU_table/',
                        replib_inputdir = os.path.abspath(OUTPUTDIR) + '/ematch_table/',
                        inswindow = INSWINDOW_FIX,
                        n_core = NCORE)

###  <font color='blue'>Filters</font> 


#### 15. Find restrict site around intergration point

In [ ]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/notfpALU_table/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_rsite/',
            refway = '/raid/retroelement/test/reference_genome/hg38_indexed_bowtie/hg38.fa',
            mseq = RESTRICT_SITE,
            mname = RESTRICT_SITE,
            shift = SHIFT_RESTRICT_SITE,
            n_core = NCORE)

#### 16. Find primer in flank

In [ ]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_rsite/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_primer/',
            refway = '/raid/retroelement/test/reference_genome/hg38_indexed_bowtie/hg38.fa',
            mseq = PRIMER,
            mname = 'MISS_P_HAMMING',
            shift = SHIFT_MISS_PRIMER,
            n_core = NCORE)

#### 17. Find part of RE in flank

In [ ]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_primer/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_re/',
            refway = '/raid/retroelement/test/reference_genome/hg38_indexed_bowtie/hg38.fa',
            mseq = None,
            mname = 'MISS_RE_HAMMING',
            shift = SHIFT_MISS_RE,
            n_core = NCORE)

#### 18. Intersect with repeats

In [ ]:
import intersection_replib
imp.reload(intersection_replib)

intersection_replib.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_re/',
                         outputdir = os.path.abspath(OUTPUTDIR) + '/filter_replib/',
                         repeatway = '/raid/retroelement/test/reference_genome/repeats_hg38.tabular',
                         n_core = NCORE)

#### 19. Apply filters

In [ ]:
import trash_deleting
imp.reload(trash_deleting)

trash_deleting.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_replib/',
                    outputdir = os.path.abspath(OUTPUTDIR) + '/pre_metatable/',
                    re_hamming = RE_HAMMING,
                    flank_errors = FLANK_ERRORS,
                    rsite_name = RESTRICT_SITE,
                    repeat = REPEAT,
                    m_primer = MISS_PRIMER,
                    primer_name = 'P',
                    m_re = MISS_RE,
                    re_name = 'RE')

###  <font color='blue'>Metatable</font> 


#### 20. Metaclustering

In [ ]:
import metacluster
imp.reload(metacluster)

metacluster.main(inputdir = os.path.abspath(OUTPUTDIR) + '/pre_metatable/',
                 outputdir = os.path.abspath(OUTPUTDIR) + '/metatable/',
                 pcdir = os.path.abspath(OUTPUTDIR) + '/new_collapse_table/',
                 target_re = RE,
                 window = WINDOW,
                 blen = BLEN)

#### 22. Template switch

In [ ]:
import template_switch
imp.reload(template_switch)

template_switch.main(inputdir = os.path.abspath(OUTPUTDIR) + '/metatable/',
                     outputdir = os.path.abspath(OUTPUTDIR) + '/result/',
                     primer = PRIMER,
                     main_flank_len = MAIN_FLANK_LEN,
                     template_switch_md = TEMPLATE_SWITCH_MD,
                     refway = '/raid/retroelement/test/reference_genome/hg38_indexed_bwa/hg38',
                     bwaway = './bwa-0.7.13/bwa')

# Result

### you can pull output table from 'result' folder in outputdir. Needed file is called 'metatable_ts.txt'